In [2]:
!pip install azureml-pipeline-wrapper==0.1.0.13487048 --extra-index-url https://azuremlsdktestpypi.azureedge.net/CLI-SDK-Runners-Validation/13487048 --user --upgrade

Looking in indexes: https://pypi.org/simple, https://azuremlsdktestpypi.azureedge.net/CLI-SDK-Runners-Validation/13487048
     |████████████████████████████████| 57 kB 275 kB/s eta 0:00:011
     |████████████████████████████████| 289 kB 468 kB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 10.5 MB/s eta 0:00:01
     |████████████████████████████████| 8.6 MB 405 kB/s eta 0:00:01
     |████████████████████████████████| 82 kB 2.0 MB/s eta 0:00:011


     |████████████████████████████████| 111 kB 4.5 MB/s eta 0:00:01


  Attempting uninstall: azureml-train-restclients-hyperdrive
    Found existing installation: azureml-train-restclients-hyperdrive 0.1.0.13479783
    Uninstalling azureml-train-restclients-hyperdrive-0.1.0.13479783:
      Successfully uninstalled azureml-train-restclients-hyperdrive-0.1.0.13479783
  Attempting uninstall: azureml-core
    Found existing installation: azureml-core 0.1.0.13479783
    Uninstalling azureml-core-0.1.0.13479783:
      Successfully uninstalled azureml-core-0.1.0.13479783
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  NOTE: The current PATH contains path(s) starting with `~`, which may not be expanded by all applications.
  Attempting uninstall: azureml-telemetry
    Found existing installation: azureml-telemetry 0.1.0.13479783
    Uninstalling azureml-telemetry-0.1.0.13479783:
      Successfully uninstalled azureml-telemetry-0.1.0.13479783
  Attempting uninstall: azureml-train-core
    Foun

In [3]:
!python --version

Python 3.6.10 :: Anaconda, Inc.


In [1]:
from azureml.core import Workspace, Run, Dataset
from azureml.pipeline.wrapper import Pipeline, dsl, Module

ws = Workspace.get(name='kubeflow_ws_1', subscription_id='74eccef0-4b8d-4f83-b5f9-fa100d155b22', resource_group='kubeflow-demo')

modulefunc = Module.load_from_yaml(ws, yaml_file='./noop/1in2out.spec.yaml')
data = Dataset.get_by_name(ws, 'training_data')

WARNING - Warning: Falling back to use azure cli login credentials.
If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [5]:
@dsl.pipeline(
    name='A pipeline composed with nodes 1 in 2 outs',
    description='A sample',
    default_compute_target='cpu-cluster'
)
def cell_division():
    layer = 5
    nodes = []
    nodes.append(modulefunc(input1=data))
    for i in range(0, layer-1):
        print('i=', i, ' nodes len=', len(nodes))
        current_layer_nodes = []
        for j in range(0, pow(2,i)):
            print('j=', j)
            n = nodes[-j-1]
            current_layer_nodes.append(modulefunc(input1=n.outputs.output1))
            current_layer_nodes.append(modulefunc(input1=n.outputs.output2))
        nodes = nodes + current_layer_nodes
    
    return {**nodes[-1].outputs}

pipeline = cell_division()

pipeline.validate(ws)

i= 0  nodes len= 1
j= 0
i= 1  nodes len= 3
j= 0
j= 1
i= 2  nodes len= 7
j= 0
j= 1
j= 2
j= 3
i= 3  nodes len= 15
j= 0
j= 1
j= 2
j= 3
j= 4
j= 5
j= 6
j= 7


{'result': 'validation passed', 'errors': []}

In [9]:
pow(2,0)

1

In [18]:
pipeline.submit_run(ws, experiment_name='samples')

Submitted PipelineRun b6684390-b2cb-4116-91ca-0fa7a509c19c
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/samples/runs/b6684390-b2cb-4116-91ca-0fa7a509c19c?wsid=/subscriptions/74eccef0-4b8d-4f83-b5f9-fa100d155b22/resourcegroups/kubeflow-demo/workspaces/kubeflow_ws_1


Experiment,Id,Type,Status,Details Page,Docs Page
samples,b6684390-b2cb-4116-91ca-0fa7a509c19c,azureml.PipelineRun,NotStarted,Link to Azure Machine Learning studio,Link to Documentation
